In [ ]:
from datetime import datetime
import os

from dasilva_space_weather_2023 import (
    load_footpoints,
    estimate_footpoint_distribution,
    plot_footpoint_distribution,
    SLIDING_WINDOW_LENGTH
)

# Check data directory exists
Data should be downloaded from Zenodo at this location, and extracted into a directory called `data/` at the top-level of this repository. The code block that follows checks that this was done.

In [ ]:
if os.path.exists('./data'):
    print('Data directory found. Good to proceed!')
else:
    raise RuntimeError(
        'The data directory does not exist. It should be downloaded and renamed '
        'to "data" at the top-level. Please do this before continuing.'
    )

# Load Source Locations (Footpoints)
This section calls the `load_footpoints()` functon to load all source locations predicted over the entire WSA time period processed. Each time we make a prediction, we will take a subset of this for the associated time window.

In [ ]:
df_fp = load_footpoints('data/March2010_ACE/PREDSOLARWIND', 'AGONG')

In [ ]:
display_cols = ['juldate', 'lon_footpoint', 'lat_footpoint']

df_fp[display_cols].head()

# Take Subset for Time Window around Prediction Time

In [ ]:
prediction_time = datetime(2010, 3, 20, 20)
start_window = prediction_time - SLIDING_WINDOW_LENGTH / 2
stop_window = prediction_time + SLIDING_WINDOW_LENGTH / 2

df_window = df_fp[(df_fp.juldate > start_window) & (df_fp.juldate < stop_window)]
df_window[display_cols].head()

In [ ]:
df_times = load_target_times('data/March2010_ACE/PREDSOLARWIND', 'AGONG')

In [ ]:
len(df_times.index)

# Generate the Probability Map
This esction uses a function `estimate_footpoint_distribution()` to compute the relative probability distribution. The core of this function uses the Scikit-Learn class [sklearn.neighbors.KernelDensity](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html). Our function returns three arrays:
- `prob_map` (2D array of relative probability values in [0, 1])
- `prob_map_lon` (the longitudes associated with `prob_map`, in radians)
- `prob_map_lat` (the latitudes associated with `prob_map`, in radians)

In [ ]:
prob_map, prob_map_lon, prob_map_lat = estimate_footpoint_distribution(prediction_time, df_window)

# Plot the Probability Map

The `plot_footpoint_distribution()` function is used to plot the distribution. This function uses [Cartopy](https://scitools.org.uk/cartopy/docs/latest/) to handle map projection and the dual longitude axis labeling. The fits file passed is used to plot the background magnetogram.

In [ ]:
plot_footpoint_distribution(
    df_window, prob_map, prob_map_lon, prob_map_lat, prediction_time,
    'Earth', 'data/March2010_ACE/CORONA', 
)